# Prompt Tuning

In [ ]:
import os
from collections import Counter

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType

import torch
from torch.utils.data import DataLoader

from tqdm import tqdm
import wandb

Make sure to login to wandb first using the cli:
```
wandb login
<paste wandb api key>
```

In [ ]:
wandb.init(project='prompt_learning_methods', name='prompt_tuning')
seed = 0
device = 'cuda'
model_name_or_path = "mistralai/Mistral-7B-v0.1"
tokenizer_name_or_path = "mistralai/Mistral-7B-v0.1"
dataset_name = "twitter_complaints"
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 1e-4
num_epochs = 10
batch_size = 8
set_seed(seed)

## Dataset prep

In [ ]:
dataset = load_dataset("ought/raft", dataset_name)
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)

dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1
)
print(dataset)
dataset["train"][0]

In [ ]:
Counter(dataset["train"]["Label"])

In [ ]:
# preprocess dataset
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(f"{target_max_length=}")

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x}\nLabel" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(input) 
    labels = tokenizer(targets, add_special_tokens=False)
    
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])

    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (max_length - len(sample_input_ids)) + model_inputs["attention_mask"][i]
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + label_input_ids
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][:max_length])
    
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
preprocessed_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset"
)

train_dataset = preprocessed_dataset["train"]
eval_dataset = preprocessed_dataset["train"]

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
next(iter(train_dataloader))

In [1]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column}: {x}\nLabel" for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    
    return model_inputs

In [ ]:
test_dataset = dataset["test"].map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset"
)
test_dataloader = DataLoader(
    test_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
next(iter(test_dataloader))

## Create PEFT model, Optimizer and LR Scheduler

In [ ]:
# prompt tunning config
# this is the general instruction
prompt_tunning_init_text = "Classify if the tweet is a complaint or no complaint. \n"
# now peft config
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=len(tokenizer(prompt_tunning_init_text)["input_ids"]),
    prompt_tuning_init_text=prompt_tunning_init_text,
    tokenizer_name_or_path=model_name_or_path
)

In [ ]:
# create model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16
)
# now homologate model to peft
model = get_peft_model(model, peft_config)
# print all trainable parameters
model.print_trainable_parameters()
# now enable gradient checkpointing for resume if stopped
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrenant": False})
model = model.to(device)

In [ ]:
# define optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.1)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)*num_epochs)
)

## Qualitative evaluation on test samples before finetuning

In [ ]:
model.eval()
i = 33
inputs = tokenizer(
    f'{text_column}: {dataset["test"][i]["Tweet text"]}\nLabel : ',
    return_tensors='pt'
)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        atytention_mask=inputs["attention_mask"],
        max_new_tokens=20,
        eos_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

## Training and evaluation loop

In [ ]:
# training and evaluation
for epoch in range(num_epochs):
    # state model on train stage
    model.train()
    # this will count loss over each epoch
    total_loss = 0
    # start iterating over each batch on train dataloader
    for step, batch in enumerate(tqdm(train_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.autocast(detype=torch.float16, device_type="cuda"):
            # map batch values to model and get outputs on forward pass
            outputs = model(**batch)
        # get loss from output
        loss = outputs.loss
        # start accumulating losses over each epoch
        total_loss += loss.detach().float()
        # do backward step
        loss.backward()
        # optimize on backward step
        optimizer.step()
        # define if learning rate must be redefine on this step
        lr_scheduler.step()
        # reset gradients for next step
        optimizer.zero_grad()

    # state model on evaluation stage
    model.eval()
    # this will count loss over each epoch for evaluation set
    eval_loss = 0
    eval_preds = []
    # start iterating over each batch on evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            # use no gradients for this cause this is eval only
            # map batch values to model and get outputs
            outputs = model(**batch)
        # get loss from evaluation outputs
        loss = outputs.loss
        # start accumulating losses over each epoch
        eval_loss += loss
        # storage predictions
        eval_preds.extend(
            tokenizer.batch_decode(
                torch.argmax(outputs.logits, -1).detach().cpu().numpy(),
                skip_special_tokens=True
            )
        )

    # get average of eval losses
    eval_epoch_loss = eval_loss / len(eval_dataloader)
    # get perpelexity of eval losses
    # perplexity: measure of how well a model is accomplishing its task.
    # read further info here: https://developers.google.com/machine-learning/glossary#perplexity
    eval_ppl = torch.exp(eval_epoch_loss)
    # get average of train losses
    train_epoch_loss = total_loss / len(train_dataloader)
    # get perpelexity of train losses
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

    # log metrics on wandb
    wandb.log(
        {
            "train": {"perplexity": train_ppl, "loss": train_epoch_loss, "epoch": epoch},
            "val": {"perplexity": eval_ppl, "loss": eval_epoch_loss, "epoch": epoch} 
        }
    )

## Qualitative evaluation on test samples after finetuning

In [ ]:
model.eval()
i = 33
inputs = tokenizer(
    f'{text_column}: {dataset["test"][i]["Tweet text"]}\nLabel: ',
    return_tensors="pt"
)
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=5,
        eos_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0])

The guide pushes the model to huggingface hub and then load it again to evaluate on test samples. </br>
```python
model.push_to_hub(
    f"mistral_prompt_tuning",
    token="hf_...",
    private=True
)
```

Or save locally:
```python
peft_model_id = "mistral_prompt_tuning",
model.save_pretrained(peft_model_id)
```

**In any case, the model checkpoint is extremelly much smaller in size than original model, which improves a lot the deployment of this model in downstream tasks.**